Copyright (c) Microsoft Corporation.  
Licensed under the MIT License.

# Abstractive Summarization using MiniLM on CNN/DailyMails

## Before you start
Set `QUICK_RUN = True` to run the notebook on a small subset of data and a smaller number of steps. If `QUICK_RUN = False`, the notebook takes about 2 hours to run on a VM with 4 16GB NVIDIA V100 GPUs. 

In [1]:
QUICK_RUN = True

## Summary
This notebook demostrates how to fine-tune the [MiniLM](https://arxiv.org/abs/2002.10957) for abstractive summarization task. Utility functions and classes in the microsoft/nlp-recipes repo are used to facilitate data preprocessing, model training, model scoring, result postprocessing, and model evaluation.

### Abstractive Summarization
Abstractive summarization is the task of taking an input text and summarizing its content in a shorter output text. In contrast to extractive summarization, abstractive summarization doesn't take sentences directly from the input text, instead, rephrases the input text.

### MiniLM
[Unified Language Model](https://arxiv.org/abs/1905.03197) (UniLM) is a state of the art model developed by Microsoft Research Asia (MSRA). The model is first pre-trained on a large unlabeled natural language corpus (English Wikipedia and BookBorpus) and can be fine-tuned on different types of labeled data for various NLP tasks like text classification and abstractive summarization. For more information, please consult the notebook [Abstractive Summarization using MiniLM on CNN/DailyMails](./abstractive_summarization_unilm_cnndm.ipynb).

Large pre-trained language models like BERT and UniLM usually consists of **hundreds** of millions of parameters and it's challleging to fine-tune such large models and also serve  real-life applications due to latency and capacity constraints.

MiniLM is a small version of UniLM, which is trained to deelply mimic UniLM with  deep self-attention knowledge distillation. It only consits of **tens** of millions of parameters (33M), which is less than one third of BERT base model and only half of the size of [DistilBERT](https://arxiv.org/abs/1910.01108). Experimental results demonstrate that MiniLM retains most of the performance of UniLM on various NLP tasks with much less computation.  


In [2]:
%load_ext autoreload
%autoreload 2
import os
import shutil
from tempfile import TemporaryDirectory
import pprint
import scrapbook as sb
import sys
import time
import torch

nlp_path = os.path.abspath("../../")
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)

from utils_nlp.dataset.cnndm import CNNDMSummarizationDatasetOrg
from utils_nlp.models.transformers.abstractive_summarization_seq2seq import S2SAbsSumProcessor, S2SAbstractiveSummarizer
from utils_nlp.eval import compute_rouge_python

from utils_nlp.models.transformers.datasets import SummarizationDataset
from utils_nlp.dataset.cnndm import detokenize

start_time = time.time()

/home/daden/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/daden/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/daden/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/daden/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: Futu

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


/home/daden/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/daden/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/daden/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/daden/.local/lib/python3.6/site-packages/tensorboard/compat/tensorf

In [3]:
# model parameters
MODEL_NAME = "minilm-l12-h384-uncased" 
MAX_SEQ_LENGTH = 512 
MAX_SOURCE_SEQ_LENGTH = 464 
MAX_TARGET_SEQ_LENGTH = MAX_SEQ_LENGTH - MAX_SOURCE_SEQ_LENGTH 

# use 0 for CPU
NUM_GPUS =  torch.cuda.device_count()

# fine-tuning parameters
TRAIN_PER_GPU_BATCH_SIZE = 4
GRADIENT_ACCUMULATION_STEPS = 1
LEARNING_RATE = 1e-4

TOP_N = -1
WARMUP_STEPS = 500
MAX_STEPS = 5000
BEAM_SIZE = 5
if QUICK_RUN:
    TOP_N = 1000
    WARMUP_STEPS = 500
    MAX_STEPS = 1000
    BEAM_SIZE = 3
    if NUM_GPUS == 0:
        TOP_N = 5
        MAX_STEPS = 10

# inference parameters
TEST_PER_GPU_BATCH_SIZE = 12
FORBID_IGNORE_WORD = "."

# mixed precision setting. To enable mixed precision training, follow instructions in SETUP.md. 
# You will be able to increase the batch sizes with mixed precision training.
FP16 = False

CLEANUP_RESULTS = False

DATA_DIR = TemporaryDirectory().name
CACHE_DIR = TemporaryDirectory().name

MODEL_DIR = "./minilm_cnndm_model"
RESULT_DIR = "./minilm_cnndm_result"
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(RESULT_DIR, exist_ok=True)
OUTPUT_FILE = os.path.join(RESULT_DIR, 'nlp_cnndm_finetuning_results.txt')

## Load the CNN/DailyMail dataset
The [CNN/DailyMail dataset](https://cs.nyu.edu/~kcho/DMQA/) was original introduced for Q&A research. There are multiple versions of the dataset processed for summarization task available on the web. The `CNNDMSummarizationDatasetOrg` function downloads a version from the [UniLM repo](https://github.com/microsoft/unilm) with minimal processing. The function returns the training and testing dataset as `SummarizationDataset` which can be further processed for model training and testing.

In [4]:
train_ds, test_ds = CNNDMSummarizationDatasetOrg(local_path=DATA_DIR, top_n=TOP_N)
print(len(train_ds))
print(len(test_ds))

1000
1000


## Preprocessing
The `S2SAbsSumProcessor` has multiple methods for converting input data in `SummarizationDataset`, `IterableSummarizationDataset` or json files into the format required for model training and testing. The preprocessing steps include
- Tokenize input text
- Convert tokens into token ids

In [5]:
processor = S2SAbsSumProcessor(model_name=MODEL_NAME,  cache_dir=CACHE_DIR)

In [6]:
train_dataset = processor.s2s_dataset_from_sum_ds(train_ds, train_mode=True)
test_dataset = processor.s2s_dataset_from_sum_ds(test_ds, train_mode=False)

100%|██████████| 1000/1000 [00:13<00:00, 72.53it/s]


In [7]:
#train_dataset = processor.s2s_dataset_from_json_or_file("/dadendev/unilm/data/xsum.train.uncased_tokenized.json", train_mode=True, top_n=TOP_N)
#test_dataset = processor.s2s_dataset_from_json_or_file("/dadendev/unilm/data/xsum.test.uncased_tokenized.json", train_mode=False, top_n=TOP_N)

In [8]:
#import torch
#torch.save(train_dataset, os.path.join(CACHE_DIR, "cnndm_train_dataset.pt"))
#torch.save(test_dataset, os.path.join(CACHE_DIR, "cnndm_test_dataset.pt"))
#train_dataset = torch.load(os.path.join(CACHE_DIR, "train_dataset.pt"))
#test_dataset = torch.load(os.path.join(CACHE_DIR, "test_dataset.pt"))

## Fine tune model

The `S2SAbstractiveSummarizer` loads a pre-trained UniLM model specified by `model_name`.  
Call `S2SAbstractiveSummarizer.list_supported_models()` to see all the supported models.  
If you want to use a model on the local disk, specify `load_model_from_dir` and `model_file_name`. This is particularly useful if you want to load a previously fine-tuned model and use it for inference directly without fine-tuning. 

In [9]:
S2SAbstractiveSummarizer.list_supported_models()

['bert-large-uncased',
 'bert-base-cased',
 'bert-large-cased',
 'roberta-base',
 'roberta-large',
 'unilm-base-cased',
 'unilm-large-cased',
 'unilm1-base-cased',
 'unilm1-large-cased',
 'unilm1.2-base-uncased',
 'minilm-l12-h384-uncased']

In [10]:
abs_summarizer = S2SAbstractiveSummarizer(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    max_source_seq_length=MAX_SOURCE_SEQ_LENGTH,
    max_target_seq_length=MAX_TARGET_SEQ_LENGTH,
    cache_dir=CACHE_DIR
)


In [11]:
abs_summarizer.model

BertForSequenceToSequence(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 384, padding_idx=0)
      (position_embeddings): Embedding(512, 384)
      (token_type_embeddings): Embedding(2, 384)
      (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=384, out_features=384, bias=True)
              (key): Linear(in_features=384, out_features=384, bias=True)
              (value): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
              (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise

In [12]:
"""
abs_summarizer = S2SAbstractiveSummarizer(
     model_name=MODEL_NAME,
     max_seq_length=MAX_SEQ_LENGTH,
     max_source_seq_length=MAX_SOURCE_SEQ_LENGTH,
    max_target_seq_length=MAX_TARGET_SEQ_LENGTH,
     load_model_from_dir=RESULT_DIR,
    model_file_name="model.5000.bin",
 )
"""

'\nabs_summarizer = S2SAbstractiveSummarizer(\n     model_name=MODEL_NAME,\n     max_seq_length=MAX_SEQ_LENGTH,\n     max_source_seq_length=MAX_SOURCE_SEQ_LENGTH,\n    max_target_seq_length=MAX_TARGET_SEQ_LENGTH,\n     load_model_from_dir=RESULT_DIR,\n    model_file_name="model.5000.bin",\n )\n'

In [13]:
#"""
abs_summarizer.fit(
    train_dataset=train_dataset,
    num_gpus=NUM_GPUS,
    per_gpu_batch_size=TRAIN_PER_GPU_BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    learning_rate=LEARNING_RATE,
    warmup_steps=WARMUP_STEPS,
    max_steps=MAX_STEPS,
    fp16=FP16,
    save_model_to_dir=MODEL_DIR
)
#"""


Iteration:   0%|          | 0/1000 [00:00<?, ?it/s]/dadendev/anaconda3/envs/nlp_gpu/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Iteration:   1%|          | 10/1000 [00:09<08:40,  1.90it/s] 

timestamp: 16/04/2020 02:17:26, average loss: 9.080113, time duration: 9.574137,
                            number of examples in current reporting: 160, step 10
                            out of total 1000


Iteration:   2%|▏         | 20/1000 [00:11<03:53,  4.19it/s]

timestamp: 16/04/2020 02:17:28, average loss: 8.981736, time duration: 2.301608,
                            number of examples in current reporting: 160, step 20
                            out of total 1000


Iteration:   3%|▎         | 30/1000 [00:14<03:42,  4.35it/s]

...

Iteration:  99%|█████████▉| 990/1000 [03:59<00:02,  4.36it/s]

timestamp: 16/04/2020 02:21:15, average loss: 2.609326, time duration: 2.286271,
                            number of examples in current reporting: 160, step 990
                            out of total 1000


Iteration: 100%|██████████| 1000/1000 [04:01<00:00,  4.36it/s]

timestamp: 16/04/2020 02:21:17, average loss: 2.579652, time duration: 2.280927,
                            number of examples in current reporting: 160, step 1000
                            out of total 1000


1000

In [14]:
#abs_summarizer.save_model(RESULT_DIR, 5000, False)

## Generate summaries on testing dataset

In [15]:
predictions = abs_summarizer.predict(
    test_dataset=test_dataset,
    num_gpus=NUM_GPUS,
    per_gpu_batch_size=TEST_PER_GPU_BATCH_SIZE,
    beam_size=BEAM_SIZE,
    max_tgt_length=MAX_TARGET_SEQ_LENGTH,
    forbid_ignore_word=FORBID_IGNORE_WORD,
    fp16=FP16
)

Evaluating: 100%|██████████| 21/21 [01:16<00:00,  3.65s/it]


In [16]:
for r in predictions[:5]:
    print(r)

french prosecutor says he ' s not aware of video video from on board the crash . new : reports of video found in the site , official says . new : official says the reports is " completely wrong "
palestinian government agreed to join the international criminal court in january . international criminal courts agreed to be members of the world ' s first court in iraq . israel , u . s . and u . s . opposed the court .
amnesty world ' s annual report says the number of death sentenced to death last year . new : report says u . s . government use the death penalty for threat of terrorism . u . s . , china , china are using the death
new : amnesty international says death penalty is " dead , " government says . pakistan , iraq , iraq is using death penalty for violence , officials say . authorities found guilty of a range of charges linked to violence in the region .
new : experts say ann frank ' s died at least a month ago . new : report : ann frank died at than least a months ago . new : r

In [17]:
test_ds.get_source()[0]

'Marseille, France (CNN) The French prosecutor leading an investigation into the crash of Germanwings Flight 9525 insisted Wednesday that he was not aware of any video footage from on board the plane. Marseille prosecutor Brice Robin told CNN that " so far no videos were used in the crash investigation . " He added, " A person who has such a video needs to immediately give it to the investigators . " Robin\'s comments follow claims by two magazines, German daily Bild and French Paris Match, of a cell phone video showing the harrowing final seconds from on board Germanwings Flight 9525 as it crashed into the French Alps . All 150 on board were killed. Paris Match and Bild reported that the video was recovered from a phone at the wreckage site. The two publications described the supposed video, but did not post it on their websites . The publications said that they watched the video, which was found by a source close to the investigation. " One can hear cries of\' My God\' in several lan

In [18]:
test_ds.get_target()[0]

'Marseille prosecutor says " so far no videos were used in the crash investigation " despite media reports. Journalists at Bild and Paris Match are " very confident " the video clip is real, an editor says. Andreas Lubitz had informed his Lufthansa training school of an episode of severe depression, airline says.'

In [19]:
predictions[0]

'french prosecutor says he \' s not aware of video video from on board the crash . new : reports of video found in the site , official says . new : official says the reports is " completely wrong "'

In [20]:
with open(OUTPUT_FILE, 'w', encoding="utf-8") as f:
    for line in predictions:
        f.write(line + '\n')

## Prediction on a single input sample

In [21]:
source = """
But under the new rule, set to be announced in the next 48 hours, Border Patrol agents would immediately return anyone to Mexico — without any detainment and without any due process — who attempts to cross the southwestern border between the legal ports of entry. The person would not be held for any length of time in an American facility.

Although they advised that details could change before the announcement, administration officials said the measure was needed to avert what they fear could be a systemwide outbreak of the coronavirus inside detention facilities along the border. Such an outbreak could spread quickly through the immigrant population and could infect large numbers of Border Patrol agents, leaving the southwestern border defenses weakened, the officials argued.
The Trump administration plans to immediately turn back all asylum seekers and other foreigners attempting to enter the United States from Mexico illegally, saying the nation cannot risk allowing the coronavirus to spread through detention facilities and Border Patrol agents, four administration officials said.
The administration officials said the ports of entry would remain open to American citizens, green-card holders and foreigners with proper documentation. Some foreigners would be blocked, including Europeans currently subject to earlier travel restrictions imposed by the administration. The points of entry will also be open to commercial traffic."""

In [22]:
singel_test_ds = SummarizationDataset(
    None, source=[source], source_preprocessing=[detokenize],
)
single_test_dataset = processor.s2s_dataset_from_sum_ds(singel_test_ds, train_mode=False)

100%|██████████| 1/1 [00:00<00:00, 171.04it/s]


In [23]:
single_prediction = abs_summarizer.predict(
    test_dataset=single_test_dataset,
    num_gpus=NUM_GPUS,
    per_gpu_batch_size=1,
    beam_size=BEAM_SIZE,
    forbid_ignore_word=FORBID_IGNORE_WORD,
    fp16=FP16
)

Evaluating: 100%|██████████| 1/1 [00:01<00:00,  1.19s/it]


In [24]:
single_prediction[0]

'u . s . border enforcement officials say some foreigners will be open to americans .'

## Evaluation
We provide utility functions for evaluating summarization models and details can be found in the [summarization evaluation notebook](./summarization_evaluation.ipynb).  
For the settings in this notebook with QUICK_RUN=False, you should get ROUGE scores close to the following numbers: <br />
``
{'rouge-1': {'f': 0.36208534811461,
             'p': 0.4743143496862804,
             'r': 0.30901813498597874},
 'rouge-2': {'f': 0.1620935174111968,
             'p': 0.2153396681546399,
             'r': 0.13747476622638555},
 'rouge-l': {'f': 0.2612394493528272,
             'p': 0.3426511372716949,
             'r': 0.22311445054693663}}
``


In [25]:
rouge_scores = compute_rouge_python(cand=predictions, ref=test_ds.get_target())
pprint.pprint(rouge_scores)

Number of candidates: 1000
Number of references: 1000
{'rouge-1': {'f': 0.2471059168881668,
             'p': 0.26124243347664683,
             'r': 0.24831050296409682},
 'rouge-2': {'f': 0.06458437980183909,
             'p': 0.06913302154728393,
             'r': 0.06444674243433718},
 'rouge-l': {'f': 0.17237949931564,
             'p': 0.18260208815440665,
             'r': 0.17339200584515088}}


In [26]:
# for testing
sb.glue("rouge_1_f_score", rouge_scores["rouge-1"]["f"])
sb.glue("rouge_2_f_score", rouge_scores["rouge-2"]["f"])
sb.glue("rouge_l_f_score", rouge_scores["rouge-l"]["f"])

## Distributed training with DistributedDataParallel (DDP)
Please consult the notebook [Abstractive Summarization using MiniLM on CNN/DailyMails](./abstractive_summarization_unilm_cnndm.ipynb) for distributed training.    

## Clean up 

In [27]:
if os.path.exists(DATA_DIR):
    shutil.rmtree(DATA_DIR, ignore_errors=True)
if os.path.exists(CACHE_DIR):
    shutil.rmtree(CACHE_DIR, ignore_errors=True)
    
if CLEANUP_RESULTS:
    if os.path.exists(MODEl_DIR):
        shutil.rmtree(MODEl_DIR, ignore_errors=True)
    if os.path.exists(RESULT_DIR):
        shutil.rmtree(RESULT_DIR, ignore_errors=True)

In [28]:
print("Total notebook running time {}".format(time.time() - start_time))

Total notebook running time 403.38391304016113
